In [ ]:
from langchain.utilities import SerpAPIWrapper

In [46]:
from langchain.chains import SequentialChain
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [45]:
# test out the wrapper
search = SerpAPIWrapper()
search.run("How is rain generated?")

'What causes rain? Clouds are made of water droplets. Within a cloud, water droplets condense onto one another, causing the droplets to grow. When these water droplets get too heavy to stay suspended in the cloud, they fall to Earth as rain.'

In [53]:
# setup something that converts my question into a good google query
template = """
You are trying to give me a most used product for {category}

Here is just the name for the product:
"""
prompt = PromptTemplate(template=template, input_variables=["category"])
llm_chain = LLMChain(prompt=prompt, llm=OpenAI(temperature=0), verbose=True, output_key="product")

category = "wallets with rfid protection"
response = llm_chain.predict(category=category)
print(response)




> Entering new LLMChain chain...
Prompt after formatting:

You are trying to give me a most used product for wallets with rfid protection

Here is just the name for the product:


> Finished chain.
Travelambo RFID Blocking Wallet.


In [55]:
# setup something that converts my question into a good google query
template = """
summarize why {product} is awesome
"""
prompt = PromptTemplate(template=template, input_variables=["product"])
llm_chain_v2 = LLMChain(prompt=prompt, llm=OpenAI(temperature=0), verbose=True, output_key="response")

overall_chain = SequentialChain(chains=[llm_chain, llm_chain_v2], 
                                input_variables=["category"],
                                output_variables=["product", "response"],
                                verbose=True)
response = overall_chain({"category":"rfid protected wallets"})
print(response["response"])



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are trying to give me a most used product for rfid protected wallets

Here is just the name for the product:


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

summarize why RFID Blocking Wallet. is awesome


> Finished chain.

> Finished chain.

RFID Blocking Wallet is an awesome product because it provides an extra layer of security for your personal information. It blocks RFID signals from being read by scanners, preventing thieves from stealing your credit card information or other sensitive data. The wallet is also lightweight and slim, making it easy to carry and store in your pocket or purse. Additionally, it is made of durable materials, ensuring it will last for years to come.


In [56]:
search = SerpAPIWrapper()
search.run("Why {product} is awesome?")

'"In this valuable book, April Dunford proves that products can be transformed by changing their context. Obviously Awesome is a must-read for any founder, ...'

In [58]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent

In [ ]:
text = """
What is the estimated net annual recurring revenue of Cisco? Using this number, calculate how many 2020 Nissan Altima S you can buy?
"""
llm = OpenAI(temperature=0)
tools = load_tools(["serpapi"], llm=llm)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
agent.run(text)

In [67]:
# SerpAI Intermediate Answer
from langchain.agents import Tool

text = """
What is the estimated net worth of Jay Z? Using this number, calculate how many 2020 Nissan Altima S you can buy?
"""

search = SerpAPIWrapper()
tool = [
    Tool(name="Intermediate Answer",
        func=search.run,
        description="")
]

llm = OpenAI(temperature=0)
# tools = load_tools(["serpapi"], llm=llm)
agent = initialize_agent(tool, llm, agent="self-ask-with-search", verbose=True)
agent.run(text)



> Entering new AgentExecutor chain...
 Yes.
Follow up: What is the estimated net worth of Jay Z?
Intermediate answer: 2.5 billion USD
Follow up: How much does a 2020 Nissan Altima S cost?
Intermediate answer: The 2020 Altima has a starting MSRP1 of $24,100 for the Altima S front-wheel drive grade and $25,450 for Altima S with Intelligent All-Wheel Drive. Both come equipped with a standard 2.5-liter engine. The 2020 Altima is also available with Nissan's advanced 2.0-liter VC-Turbo engine (front-wheel drive only).
So the final answer is: You can buy 104,166 2020 Nissan Altima S with Jay Z's estimated net worth of 2.5 billion USD.

> Finished chain.


"You can buy 104,166 2020 Nissan Altima S with Jay Z's estimated net worth of 2.5 billion USD."

ToDo: build a custom agent

ToDo: do document loading

ToDo: add memory